In [213]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import *
from sklearn.pipeline import *
from sklearn.metrics import *
import pickle
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

Препроцессинг

In [214]:
raw_path = '../data/raw/'
train = pd.read_csv(os.path.join(raw_path, 'train.csv'))
test = pd.read_csv(os.path.join(raw_path, 'test.csv'))

In [215]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
SMOKE_FREQ = 'Частота пасс кур'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Частота пасс кур', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Пол', 'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког']

In [216]:
def set_idx(df: pd.DataFrame, idx_col: str) -> pd.DataFrame:
    df = df.set_index(idx_col)
    return df

def drop_unnecesary_id(df: pd.DataFrame) -> pd.DataFrame:
    if 'ID_y' in df.columns:
        df = df.drop('ID_y', axis=1)
    return df

def fill_sex(df: pd.DataFrame) -> pd.DataFrame:
    most_freq = df[SEX_COL].value_counts().index[0]
    df[SEX_COL] = df[SEX_COL].fillna(most_freq)
    return df

def fill_smoke_freq(df: pd.DataFrame) -> pd.DataFrame:
    df[SMOKE_FREQ] = df[SMOKE_FREQ].fillna('0')
    return df

def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df[CAT_COLS] = df[CAT_COLS].astype('category')

    ohe_int_cols = df[OHE_COLS].select_dtypes('number').columns
    df[ohe_int_cols] = df[ohe_int_cols].astype(np.int8)

    df[REAL_COLS] = df[REAL_COLS].astype(np.float32)

    df[SMOKE_FREQ] = df[SMOKE_FREQ].astype(object)
    return df

In [217]:
gender = {
    'Ж': 0, 'М': 1
}
def gender_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df[SEX_COL] = df[SEX_COL].map(gender).astype(np.int8)
    return df

family = {
    'никогда не был(а) в браке': 0, 'в браке в настоящее время': 1, 'гражданский брак / проживание с партнером': 2, 'в разводе': 3, 'вдовец / вдова': 4, 'раздельное проживание (официально не разведены)': 5
}
def family_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Семья'] = df['Семья'].map(family).astype(np.int8)
    return df

ethnos = {
    'другая азиатская (Корея, Малайзия, Таиланд, Вьетнам, Казахстан, Киргизия, Туркмения, Узбекистан, Таджикистан)': 0, 'европейская': 1, 'прочее (любая иная этно-расовая группа, не представленная выше)': 2
}
def ethnos_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Этнос'] = df['Этнос'].map(ethnos).astype(np.int8)
    return df

nation = {
    'Русские': 0, 'Татары': 1, 'Немцы': 2, 'Азербайджанцы': 3, 'Эстонцы': 4, 'Молдаване': 5, 'Украинцы':6, 'Чуваши': 7, 'Мордва': 8, 'Киргизы': 9, 'Казахи': 10, 'Армяне': 11, 'Белорусы': 12, 'Таджики': 13, 'Башкиры': 14, 'Евреи': 15, 'Буряты': 16, 'Удмурты': 17, 'Лезгины':18, 'Другие национальности': 19
}
def nation_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Национальность'] = df['Национальность'].map(nation).astype(np.int8)
    return df

religion = {
    'Атеист / агностик': 0, 'Христианство': 1, 'Ислам': 2, 'Индуизм': 3, 'Другое': 4, 'Нет': 5
}
def religion_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Религия'] = df['Религия'].map(religion).astype(np.int8)
    return df

def add_ord_edu(df: pd.DataFrame) -> pd.DataFrame:
    df[EDU_COL] = df[EDU_COL].str.slice(0, 1).astype(np.int8).values
    return df

profession = {
    'ведение домашнего хозяйства': 0, 'служащие': 1, 'работники,  занятые в сфере обслуживания, торговые работники магазинов и рынков': 2, 'низкоквалифицированные работники': 3, 'дипломированные специалисты': 4, 'операторы и монтажники установок и машинного оборудования': 5, 'представители   законодат.   органов   власти,  высокопостав. долж.лица и менеджеры': 6, 'техники и младшие специалисты': 7, 'квалифицированные работники сельского хозяйства и рыболовного': 8, 'ремесленники и представители других отраслей промышленности': 9, 'вооруженные силы': 10
}
def profession_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Профессия'] = df['Профессия'].map(profession).astype(np.int8)
    return df

param_smoke = {
    'Курит': 0, 'Бросил(а)': 1, 'Никогда не курил(а)': 2, 'Никогда не курил': 3
}
def param_smoke_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Статус Курения'] = df['Статус Курения'].map(param_smoke).astype(np.int8)
    return df

#passive_smoking_frequency = {
#    '0': 0, 'не менее 1 раза в день': 1, '2-3 раза в день': 2, '4 и более раз в день': 3, '1-2 раза в неделю': 4, '3-6 раз в неделю': 5
#}
#def passive_smoking_frequency_to_num(df: pd.DataFrame) -> pd.DataFrame:
#    df[SMOKE_FREQ] = df[SMOKE_FREQ].map(passive_smoking_frequency).astype(np.int8)

alcohol = {
    'никогда не употреблял': 0, 'ранее употреблял': 1, 'употребляю в настоящее время': 2
}
def alcohol_to_num(df: pd.DataFrame) -> pd.DataFrame:
    df['Алкоголь'] = df['Алкоголь'].map(alcohol).astype(np.int8)
    return df

In [218]:
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    df = set_idx(df, ID_COL)
    df = drop_unnecesary_id(df)
    df = fill_sex(df)
    df = fill_smoke_freq(df)
    df = cast_types(df)

    df = gender_to_num(df)
    
    df = family_to_num(df)
    
    df = ethnos_to_num(df)
    
    df = nation_to_num(df)
    
    df = religion_to_num(df)
    
    df = add_ord_edu(df)
    
    df = profession_to_num(df)
    
    df = param_smoke_to_num(df)
    
    #df = passive_smoking_frequency_to_num(df)
    
    df = alcohol_to_num(df)
    return df

In [219]:
train = preprocess_data(train)
test = preprocess_data(test)

In [220]:
pd.set_option('display.max_columns', None)
train

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,Сахарный диабет,Гепатит,Онкология,Хроническое заболевание легких,Бронжиальная астма,Туберкулез легких,ВИЧ/СПИД,Регулярный прим лекарственных средств,Травмы за год,Переломы,Статус Курения,Возраст курения,Сигарет в день,Пассивное курение,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,1,1,1,0,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,15.0,20.0,0,0,2,18.0,22:00:00,06:00:00,0,0,0,0,0,0,0,0
54-103-101-01,0,3,1,0,1,5,4,0,0,0,1,0,0,0,0,0,0,1,0,1,2,NaN,NaN,0,0,0,NaN,00:00:00,04:00:00,1,0,0,1,1,0,0,0
54-501-026-03,0,1,1,0,1,5,4,0,0,0,0,0,0,0,0,0,0,1,0,0,2,NaN,NaN,1,1-2 раза в неделю,2,17.0,23:00:00,07:00:00,0,0,0,0,0,0,0,0
54-501-094-02,1,1,1,0,0,3,3,1,0,0,0,0,1,0,0,0,0,1,0,0,1,12.0,10.0,1,3-6 раз в неделю,2,13.0,23:00:00,07:00:00,0,0,0,1,0,0,0,0
54-503-022-01,0,1,1,0,1,3,5,0,0,1,1,1,0,0,0,0,0,1,0,1,2,NaN,NaN,1,не менее 1 раза в день,2,16.0,23:00:00,06:00:00,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54-103-006-01,1,1,1,0,1,4,5,1,0,0,0,0,0,0,0,0,0,0,0,1,0,16.0,20.0,0,0,2,18.0,22:00:00,06:00:00,0,0,0,0,0,0,0,0
54-701-039-01,0,4,1,0,1,3,3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,15.0,10.0,0,0,1,26.0,21:30:00,07:15:00,0,0,0,0,0,0,0,0
54-002-014-01,0,1,1,0,1,3,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,20.0,10.0,0,0,2,20.0,22:30:00,08:00:00,1,0,0,1,0,1,0,0


MODELING

In [221]:
processed_data_path = '../data/processed/'
train = pd.read_pickle(os.path.join(processed_data_path, 'train.pkl'))
target = pd.read_pickle(os.path.join(processed_data_path, 'target.pkl'))

In [222]:
train_data, val_data, train_target, val_target = train_test_split(train, target, train_size=0.8, random_state=64)

In [223]:
def predict_model(model: MultiOutputClassifier,
            train_data: pd.DataFrame,
            val_data: pd.DataFrame,
            train_target: pd.DataFrame,
            val_target: pd.DataFrame):
    val_target_pred = model.predict(val_data)
    train_target_pred = model.predict(train_data)

    classif_train = multilabel_confusion_matrix(train_target_pred, train_target)
    TrueNegative = []
    FalsePositive = []
    FalseNegative = []
    TruePositive = []
    precision = []
    for matr in classif_train:
        tn, fp, fn, tp = matr.ravel()
        TrueNegative.append(tn)
        FalsePositive.append(fp)
        FalseNegative.append(fn)
        TruePositive.append(tp)
        precision.append(sum(TruePositive) / (sum(TruePositive) + sum(FalsePositive)))

    TrueNegative = []
    FalsePositive = []
    FalseNegative = []
    TruePositive = []
    recall = []
    classif_test = multilabel_confusion_matrix(val_target_pred, val_target)
    for matr in classif_test:
        tn, fp, fn, tp = matr.ravel()
        TrueNegative.append(tn)
        FalsePositive.append(fp)
        FalseNegative.append(fn)
        TruePositive.append(tp)
        recall.append(sum(TruePositive) / (sum(TruePositive) + sum(FalseNegative)))
    
    return precision, recall

In [224]:
def train_model(train_data: pd.DataFrame, train_target: pd.DataFrame) -> MultiOutputClassifier:
    cat_model = CatBoostClassifier(n_estimators=100,
                                   depth=8,
                                   learning_rate=1, 
                                   loss_function='Logloss')
    model = MultiOutputClassifier(cat_model, n_jobs=1)
    model.fit(train_data, train_target)
    
    return model

In [225]:
model = train_model(train_data, train_target)
precision, recall = predict_model(model, train_data, val_data, train_target, val_target)

CatBoostError: features data: pandas.DataFrame column 'Частота пасс кур' has dtype 'category' but is not in  cat_features list

In [ ]:
print(precision)
print(recall)

In [ ]:
final_pred = model.predict(train)
final_pred = pd.DataFrame(data=final_pred, columns=TARGET_COLS)

In [ ]:
final_pred

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
ID,,,,,
54-102-358-02,0,0,0,0,0
54-103-101-01,1,1,0,0,0
54-501-026-03,0,0,0,0,0
54-501-094-02,1,0,0,0,0
54-503-022-01,1,0,1,1,0
...,...,...,...,...,...
54-103-006-01,0,0,0,0,0
54-701-039-01,0,0,0,0,0
54-002-014-01,1,0,1,0,0
